In [1]:
try:
    import blissdedrift
except:
    print("Failed to install bliss from installed module. Attempting build directory development module")
    import sys
    sys.path.append("../build/bliss/python")
    import blissdedrift


Failed to install bliss from installed module. Attempting build directory development module
Found a usable device


<frozen importlib._bootstrap>:488: RuntimeWarning: nanobind: type 'ndarray' was already registered!

<frozen importlib._bootstrap>:488: RuntimeWarning: nanobind: type 'dev' was already registered!

<frozen importlib._bootstrap>:488: RuntimeWarning: nanobind: type 'datatype' was already registered!



In [2]:
import h5py
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt

import scipy.stats as spstat
from pprint import pp


In [3]:
filpath = "/home/nathan/datasets/voyager_2020_data/single_coarse_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.h5"

In [4]:
noise_stats = {}

In [5]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cpu_cc = np.from_dlpack(cc.data)


noise_power_baseline = np.std(cpu_cc)
noise_floor_baseline = np.mean(cpu_cc)

noise_stats["baseline"] = {
    "power": noise_power_baseline,
    "floor": noise_floor_baseline
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [6]:

noise_slice = cpu_cc[:,250000:500000]
noise_power_slice = np.std(noise_slice)
noise_floor_slice = np.mean(noise_slice)

noise_stats["slice"] = {
    "power": noise_power_slice,
    "floor": noise_floor_slice
}


In [7]:
vals, lower, upper = spstat.sigmaclip(cpu_cc)

noise_power_vals = np.std(vals)
noise_floor_vals = np.mean(vals)

noise_stats["sigmaclip_baseline"] = {
    "power": noise_power_vals,
    "floor": noise_floor_vals
}



In [8]:
pp(noise_stats)

{'baseline': {'power': 3570070.5, 'floor': 5140684.0},
 'slice': {'power': 564954.3, 'floor': 5504788.0},
 'sigmaclip_baseline': {'power': 936291.56, 'floor': 5136734.0}}


In [9]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

# cc.noise_estimate(cc, noise_est_options)
cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc, noise_est_options)

noise_stats["bliss_baseline"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [10]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_standard_flag = blissdedrift.flaggers.flag_spectral_kurtosis(cc, .05, 25)

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_standard_flag, noise_est_options)

noise_stats["bliss_sk"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51


In [11]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_standard_flag = blissdedrift.flaggers.flag_filter_rolloff(cc, .25)

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_standard_flag, noise_est_options)

noise_stats["bliss_rolloff"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [12]:

sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_sigmaclip = blissdedrift.flaggers.flag_sigmaclip(cc, 35, 5, 5)


noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_sigmaclip, noise_est_options)

noise_stats["bliss_sigmaclip"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
iter 0:  mean=5140680.5    std=3570070.2
iter 1:  mean=5137039.5    std=937480.7
iter 2:  mean=5136866.5    std=936565.9
iter 3:  mean=5136866.5    std=936564.75


In [13]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_standard_flag = blissdedrift.flaggers.flag_filter_rolloff(cc, .2)
cc_standard_flag = blissdedrift.flaggers.flag_spectral_kurtosis(cc_standard_flag, .05, 25)

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_standard_flag, noise_est_options)

noise_stats["bliss_rolloff_sk"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51


In [14]:

sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_sigmaclip = blissdedrift.flaggers.flag_filter_rolloff(cc, .2)
cc_sigmaclip = blissdedrift.flaggers.flag_spectral_kurtosis(cc_sigmaclip, .05, 25)
cc_sigmaclip = blissdedrift.flaggers.flag_sigmaclip(cc_sigmaclip, 35, 5, 5)


noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_sigmaclip, noise_est_options)

noise_stats["bliss_rolloff_sk_sigmaclip"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51
iter 0:  mean=5140680.5    std=3570070.2
iter 1:  mean=5137039.5    std=937480.7
iter 2:  mean=5136866.5    std=936565.9
iter 3:  mean=5136866.5    std=936564.75


In [15]:

sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc.set_device("cuda:0")

cc_sigmaclip = blissdedrift.flaggers.flag_spectral_kurtosis(cc, .05, 25)
cc_sigmaclip = blissdedrift.flaggers.flag_sigmaclip(cc_sigmaclip, 35, 5, 5)


noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_sigmaclip, noise_est_options)

noise_stats["bliss_sk_sigmaclip"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51
iter 0:  mean=5140680.5    std=3570070.2
iter 1:  mean=5137039.5    std=937480.7
iter 2:  mean=5136866.5    std=936565.9
iter 3:  mean=5136866.5    std=936564.75


In [16]:
pp(noise_stats)

{'baseline': {'power': 3570070.5, 'floor': 5140684.0},
 'slice': {'power': 564954.3, 'floor': 5504788.0},
 'sigmaclip_baseline': {'power': 936291.56, 'floor': 5136734.0},
 'bliss_baseline': {'power': 3570070.25, 'floor': 5140680.5},
 'bliss_sk': {'power': 937606.875, 'floor': 5137023.0},
 'bliss_rolloff': {'power': 4904291.0, 'floor': 5502934.5},
 'bliss_sigmaclip': {'power': 936564.75, 'floor': 5136866.5},
 'bliss_rolloff_sk': {'power': 569054.625, 'floor': 5497296.5},
 'bliss_rolloff_sk_sigmaclip': {'power': 566329.375, 'floor': 5497009.5},
 'bliss_sk_sigmaclip': {'power': 936558.6875, 'floor': 5136842.5}}


In [17]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cpu_cc = np.from_dlpack(cc.data)


noise_power_baseline = np.std(cpu_cc)
noise_floor_baseline = np.mean(cpu_cc)

noise_stats["corrected_baseline"] = {
    "power": noise_power_baseline,
    "floor": noise_floor_baseline
}


noise_slice = cpu_cc[:,250000:500000]
noise_power_slice = np.std(noise_slice)
noise_floor_slice = np.mean(noise_slice)

noise_stats["corrected_slice"] = {
    "power": noise_power_slice,
    "floor": noise_floor_slice
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [18]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cc.set_device("cuda:0")

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

# cc.noise_estimate(cc, noise_est_options)
cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc, noise_est_options)

noise_stats["bliss_corrected_baseline"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}





INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [19]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cc.set_device("cuda:0")

cc_standard_flag = blissdedrift.flaggers.flag_spectral_kurtosis(cc, .05, 25)

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_standard_flag, noise_est_options)

noise_stats["bliss_corrected_sk"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51


In [20]:
sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cc.set_device("cuda:0")

cc_standard_flag = blissdedrift.flaggers.flag_filter_rolloff(cc, .20)

noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_standard_flag, noise_est_options)

noise_stats["bliss_corrected_rolloff"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '


In [21]:

sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cc.set_device("cuda:0")

cc_sigmaclip = blissdedrift.flaggers.flag_sigmaclip(cc, 35, 5, 5)


noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_sigmaclip, noise_est_options)

noise_stats["bliss_corrected_sigmaclip"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
iter 0:  mean=5521440    std=3521909.8
iter 1:  mean=5517768.5    std=569019.6
iter 2:  mean=5517341.5    std=566825.94
iter 3:  mean=5517338    std=566811.9
iter 4:  mean=5517337.5    std=566814.9


In [22]:

sc = blissdedrift.scan(filpath)
cc = sc.read_coarse_channel(0)
cc = blissdedrift.preprocess.equalize_passband_filter(cc, "../gbt_pfb_response.f32")
cc.set_device("cuda:0")

cc_sigmaclip = blissdedrift.flaggers.flag_spectral_kurtosis(cc, .05, 25)
cc_sigmaclip = blissdedrift.flaggers.flag_sigmaclip(cc_sigmaclip, 35, 5, 5)


noise_est_options = blissdedrift.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True

cc_noise_est = blissdedrift.estimators.estimate_noise_power(cc_sigmaclip, noise_est_options)

noise_stats["bliss_corrected_sk_sigmaclip"] = {
    "power": cc_noise_est.noise_power,
    "floor": cc_noise_est.noise_floor
}


INFO: HDF5 looking for filter plugins in: 
	'/usr/lib/x86_64-linux-gnu/hdf5/serial/plugin '
INFO: spec kurtosis with M=16 and N=51
iter 0:  mean=5521440    std=3521909.8
iter 1:  mean=5517768.5    std=569019.6
iter 2:  mean=5517341.5    std=566825.94
iter 3:  mean=5517338    std=566811.9
iter 4:  mean=5517337.5    std=566814.9


In [23]:
pp(noise_stats)

{'baseline': {'power': 3570070.5, 'floor': 5140684.0},
 'slice': {'power': 564954.3, 'floor': 5504788.0},
 'sigmaclip_baseline': {'power': 936291.56, 'floor': 5136734.0},
 'bliss_baseline': {'power': 3570070.25, 'floor': 5140680.5},
 'bliss_sk': {'power': 937606.875, 'floor': 5137023.0},
 'bliss_rolloff': {'power': 4904291.0, 'floor': 5502934.5},
 'bliss_sigmaclip': {'power': 936564.75, 'floor': 5136866.5},
 'bliss_rolloff_sk': {'power': 569054.625, 'floor': 5497296.5},
 'bliss_rolloff_sk_sigmaclip': {'power': 566329.375, 'floor': 5497009.5},
 'bliss_sk_sigmaclip': {'power': 936558.6875, 'floor': 5136842.5},
 'corrected_baseline': {'power': 3521909.5, 'floor': 5521433.0},
 'corrected_slice': {'power': 567031.25, 'floor': 5527145.0},
 'bliss_corrected_baseline': {'power': 3521909.75, 'floor': 5521440.0},
 'bliss_corrected_sk': {'power': 569222.6875, 'floor': 5517762.0},
 'bliss_corrected_rolloff': {'power': 4523199.5, 'floor': 5526402.5},
 'bliss_corrected_sigmaclip': {'power': 566814.8

In [24]:
from IPython.display import display, Math



In [25]:

print("\t\t\t\t|rolloff| SK\t|σ-clip\t|pfbcomp| power     | floor\t")
print("-----------------------------------------------------------")
print(f"full spectrum baseline \t\t| \t| \t| \t| \t| {noise_stats["baseline"]["power"]:9.1f} | {noise_stats["baseline"]["floor"]:9.1f}")
print(f"slice noise baseline \t\t| \t| \t| \t| \t| {noise_stats["slice"]["power"]:9.1f} | {noise_stats["slice"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| \t| ✓\t| \t| {noise_stats["sigmaclip_baseline"]["power"]:9.1f} | {noise_stats["sigmaclip_baseline"]["floor"]:8.1f}")

print(f"bliss baseline \t\t\t| \t| \t| \t| \t| {noise_stats["bliss_baseline"]["power"]:9.1f} | {noise_stats["bliss_baseline"]["floor"]:8.1f}")
print(f" \t\t\t\t| ✓\t| \t| \t| \t| {noise_stats["bliss_rolloff"]["power"]:9.1f} | {noise_stats["bliss_rolloff"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| ✓\t| \t| \t| {noise_stats["bliss_sk"]["power"]:9.1f} | {noise_stats["bliss_sk"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| \t| ✓\t| \t| {noise_stats["bliss_sigmaclip"]["power"]:9.1f} | {noise_stats["bliss_sigmaclip"]["floor"]:8.1f}")
print(f" \t\t\t\t| ✓\t| ✓\t| \t| \t| {noise_stats["bliss_rolloff_sk"]["power"]:9.1f} | {noise_stats["bliss_rolloff_sk"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| ✓\t| ✓\t| \t| {noise_stats["bliss_sk_sigmaclip"]["power"]:9.1f} | {noise_stats["bliss_sk_sigmaclip"]["floor"]:8.1f}")
print(f" \t\t\t\t| ✓\t| ✓\t| ✓\t| \t| {noise_stats["bliss_rolloff_sk_sigmaclip"]["power"]:9.1f} | {noise_stats["bliss_rolloff_sk_sigmaclip"]["floor"]:8.1f}")

print("-----------------------------------------------------------")

print(f"corrected spectrum baseline \t| \t| \t| \t| ✓\t| {noise_stats["corrected_baseline"]["power"]:9.1f} | {noise_stats["corrected_baseline"]["floor"]:8.1f}")
print(f"corrected spectrum slice \t| \t| \t| \t| ✓\t| {noise_stats["corrected_slice"]["power"]:9.1f} | {noise_stats["corrected_slice"]["floor"]:8.1f}")
print(f"corrected spectrum bliss \t| \t| \t| \t| ✓\t| {noise_stats["bliss_corrected_baseline"]["power"]:9.1f} | {noise_stats["bliss_corrected_baseline"]["floor"]:8.1f}")
print(f" \t\t\t\t| ✓\t| \t| \t| ✓\t| {noise_stats["bliss_corrected_rolloff"]["power"]:9.1f} | {noise_stats["bliss_corrected_rolloff"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| ✓\t| \t| ✓\t| {noise_stats["bliss_corrected_sk"]["power"]:9.1f} | {noise_stats["bliss_corrected_sk"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| \t| ✓\t| ✓\t| {noise_stats["bliss_corrected_sigmaclip"]["power"]:9.1f} | {noise_stats["bliss_corrected_sigmaclip"]["floor"]:8.1f}")
print(f" \t\t\t\t| \t| ✓\t| ✓\t| ✓\t| {noise_stats["bliss_corrected_sk_sigmaclip"]["power"]:9.1f} | {noise_stats["bliss_corrected_sk_sigmaclip"]["floor"]:8.1f}")





				|rolloff| SK	|σ-clip	|pfbcomp| power     | floor	
-----------------------------------------------------------
full spectrum baseline 		| 	| 	| 	| 	| 3570070.5 | 5140684.0
slice noise baseline 		| 	| 	| 	| 	|  564954.3 | 5504788.0
 				| 	| 	| ✓	| 	|  936291.6 | 5136734.0
bliss baseline 			| 	| 	| 	| 	| 3570070.2 | 5140680.5
 				| ✓	| 	| 	| 	| 4904291.0 | 5502934.5
 				| 	| ✓	| 	| 	|  937606.9 | 5137023.0
 				| 	| 	| ✓	| 	|  936564.8 | 5136866.5
 				| ✓	| ✓	| 	| 	|  569054.6 | 5497296.5
 				| 	| ✓	| ✓	| 	|  936558.7 | 5136842.5
 				| ✓	| ✓	| ✓	| 	|  566329.4 | 5497009.5
-----------------------------------------------------------
corrected spectrum baseline 	| 	| 	| 	| ✓	| 3521909.5 | 5521433.0
corrected spectrum slice 	| 	| 	| 	| ✓	|  567031.2 | 5527145.0
corrected spectrum bliss 	| 	| 	| 	| ✓	| 3521909.8 | 5521440.0
 				| ✓	| 	| 	| ✓	| 4523199.5 | 5526402.5
 				| 	| ✓	| 	| ✓	|  569222.7 | 5517762.0
 				| 	| 	| ✓	| ✓	|  566814.9 | 5517337.5
 				| 	| ✓	| ✓	| ✓	|  566800.4 |

In [26]:
display(Math(r"\checkmark"))

<IPython.core.display.Math object>

In [27]:
3570070.5/3521909.5

1.013674684145064

In [28]:
p = 1/16 777 216
p


SyntaxError: invalid syntax (1676554632.py, line 1)

In [ ]:
from scipy.special import erfc

def qfunc(x):
    return 0.5 * erfc(x / np.sqrt(2))

x = 6
result = qfunc(x)
print(result)